In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, isnull, lit, length, explode, count, upper, lower, regexp_replace, regexp_extract

In [18]:
from pyspark.sql import SparkSession
# enableVectorizedReader is set to false to avoid issues with certain Parquet files
spark = SparkSession.builder \
    .appName("Data quality check vendas") \
    .config("spark.sql.parquet.enableVectorizedReader", "false") \
    .config("spark.hadoop.io.nativeio", "false") \
    .getOrCreate()

In [19]:
df = spark.read.parquet("/app/data/gold/vendas.parquet")

In [20]:
df.printSchema()

root
 |-- codigo_filial: long (nullable = true)
 |-- codigo_cupom_venda: long (nullable = true)
 |-- data_emissao: timestamp (nullable = true)
 |-- codigo_item: long (nullable = true)
 |-- valor_unitario: decimal(38,2) (nullable = true)
 |-- quantidade: decimal(38,4) (nullable = true)
 |-- codigo_cliente: string (nullable = true)
 |-- tipo_desconto: string (nullable = true)
 |-- canal_venda: string (nullable = true)



In [21]:
df.show(5)

+-------------+------------------+-------------------+-----------+--------------+----------+--------------------+-------------+-----------+
|codigo_filial|codigo_cupom_venda|       data_emissao|codigo_item|valor_unitario|quantidade|      codigo_cliente|tipo_desconto|canal_venda|
+-------------+------------------+-------------------+-----------+--------------+----------+--------------------+-------------+-----------+
|      2711884|            350031|2023-12-26 00:00:00|  478225826|          5.99|    1.0000|03B9CB3A83D23AFB2112|          SIM|          L|
|      2711884|            350031|2023-12-26 00:00:00|  403396526|          3.99|    1.0000|03B9CB3A83D23AFB2112|          SIM|          L|
|      2711884|            350031|2023-12-26 00:00:00|  501265726|          5.59|    1.0000|03B9CB3A83D23AFB2112|          SIM|          L|
|      2711884|            350031|2023-12-26 00:00:00|  403649896|         59.80|    2.0000|03B9CB3A83D23AFB2112|          NAO|          L|
|      2711884|     

In [22]:
df.select([
    count(when(isnull(c), c)).alias(c) for c in df.columns
]).show()

+-------------+------------------+------------+-----------+--------------+----------+--------------+-------------+-----------+
|codigo_filial|codigo_cupom_venda|data_emissao|codigo_item|valor_unitario|quantidade|codigo_cliente|tipo_desconto|canal_venda|
+-------------+------------------+------------+-----------+--------------+----------+--------------+-------------+-----------+
|            0|                 0|           0|          0|             0|         0|             0|            0|       9505|
+-------------+------------------+------------+-----------+--------------+----------+--------------+-------------+-----------+



In [24]:
spark.stop()
